In [1]:
from regulus.utils import io
from regulus.topo import * 
from regulus.alg import *
from regulus.measures import *
from regulus.models import *
from regulus.tree import *

from ipyregulus import TreeWidget, TreeView

from ipyregulus import *

from sidepanel import SidePanel

In [2]:
gauss = io.load('data/gauss4.p')
gauss.add_attr('linear', node_model(linear_model))
gauss.add_attr('fitness', fitness)
gauss.add_attr('relative_fitness', relative_fitness)
gauss.tree.add_attr('parent_fitness', parent_fitness)
gauss.tree.add_attr('child_fitness', child_fitness)
gauss.tree.add_attr('size', node_size)
gauss.tree.add_attr('rel_size', node_relative_size)
gauss.tree.add_attr('span', node_span)

In [3]:
from ipywidgets import Output
debug_view = Output(title='debug')
debug_view

Output()

In [4]:
def update(view):
    def f(show):
        view.show = show
    return f

In [5]:
w1 = TreeWidget(tree=gauss.tree)

In [9]:
v1 = TreeView(w1, attr='fitness')

In [10]:
v1

TreeView(field='fitness', tree_model=TreeWidget(attrs={'fitness': {0: 0.37445782174461234, 1: 0.38545663840155…

In [8]:
f1 = TreeFilter(v1.tree, 'fitness', lambda a,b: a>=b)

In [9]:
f1.on(update(v1))

In [10]:
f1

In [11]:
# w2 = TreeWidget(gauss.tree)

In [12]:
v2 = TreeView(gauss.tree, measure='span')

In [13]:
f2 = TreeFilter(v2.tree, 'span',  lambda a,b: a>=b)

In [14]:
f2.on(update(v2))

In [15]:
p2 = SidePanel(title='TreeView 2')
with p2:
    display(v2, f2)

In [19]:
p1 = SidePanel(title='TreeView 1')
with p1:
    display(v1, f1)

In [21]:
f2.off(None)

In [22]:
f2.on(update(v1))

In [17]:
f2.on(update)

In [23]:
with p1:
    display(v2)

In [24]:
f2.on(update(v2))